In [9]:
import sqlite3
import pandas as pd
connection = sqlite3.connect('./data//sql-murder-mystery.db')
cursor_challenge=connection.cursor()

In [ ]:
#para poder ver toda la información ajustamos la cantidad de carácteres que se ven por celda
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
#definimos función para evitar escribir el mismo código muchas veces
def sql_query(query):
    cursor_challenge.execute(query)
    ans = cursor_challenge.fetchall()
    names = [description[0] for description in cursor_challenge.description]
    return pd.DataFrame(ans,columns=names)

In [ ]:
#vemos la estructura de los datos, para ver más o menos donde podemos buscar la información
query='''
SELECT name 
  FROM sqlite_master
 where type = 'table'
 '''
sql_query(query)

,name
0,crime_scene_report
1,drivers_license
2,person
3,facebook_event_checkin
4,interview
5,get_fit_now_member
6,get_fit_now_check_in
7,income
8,solution


In [29]:
#buscamos el caso con las pistas que recordamos
query = '''
SELECT *
FROM crime_scene_report
WHERE
    type = 'murder' AND
    date = 20180115 AND
    city = 'SQL City';
'''
sql_query(query)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [25]:
#primer testigo
query1='''
SELECT *
FROM person
WHERE address_street_name = 'Northwestern Dr'
ORDER BY address_number DESC
LIMIT 1;
'''
#segundo testigo
query2='''
SELECT *
FROM person
WHERE name LIKE 'Annabel%' AND address_street_name = 'Franklin Ave'
'''
print(sql_query(query1))
print(sql_query(query2))

      id            name  license_id  address_number address_street_name  \
0  14887  Morty Schapiro      118009            4919     Northwestern Dr   

         ssn  
0  111564949  
      id            name  license_id  address_number address_street_name  \
0  16371  Annabel Miller      490173             103        Franklin Ave   

         ssn  
0  318771143  


In [30]:
#leemos sus entrevistas
query='''
SELECT *
FROM interview
WHERE person_id IN (14887, 16371)
'''
sql_query(query)

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."
1,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [75]:
#ahora vamos a buscar a la persona cuyo membership pueda encajar con la descripción del testigo 1, la guardamos
query='''
SELECT *
FROM get_fit_now_member
WHERE id LIKE '48Z%' AND membership_status = 'gold' 
'''
suspects= sql_query(query)
print(suspects)
#vemos si una de las personas se encontraba en el gimnasio el día que dijo verlo el segundo testigo
query='''
SELECT *
FROM get_fit_now_check_in
WHERE membership_id IN ('48Z7A', '48Z55') AND check_in_date = 20180109
'''

suspects2_0=sql_query(query)
print("\n")
print(suspects2_0)

      id  person_id           name  membership_start_date membership_status
0  48Z7A      28819   Joe Germuska               20160305              gold
1  48Z55      67318  Jeremy Bowers               20160101              gold


  membership_id  check_in_date  check_in_time  check_out_time
0         48Z7A       20180109           1600            1730
1         48Z55       20180109           1530            1700


In [77]:
#ahora vamos a revisar si alguno tiene un coche parecido al descrito, ya que ambos sospechosos fueron al gimnasio ese dia
query='''
SELECT *
FROM drivers_license
WHERE plate_number LIKE '%H42W%'
'''
suspects3_0= sql_query(query)
print(suspects3_0)

#comprobamos si una de estas tres personas se corresponden con los ya sospechosos
query='''
SELECT *
FROM person
WHERE license_id IN ('183779', '423327','664760')
'''
print("\n")
sql_query(query)

       id  age  height eye_color hair_color  gender plate_number   car_make car_model
0  183779   21      65      blue     blonde  female       H42W0X     Toyota     Prius
1  423327   30      70     brown      brown    male       0H42W2  Chevrolet  Spark LS
2  664760   21      71     black      black    male       4H42WR     Nissan    Altima




,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
2,78193,Maxine Whitely,183779,110,Fisk Rd,137882671


El asesino es **JEREMY BOWERS**, todos los resultados coinciden, tanto el coche, como la membresía.